这段代码展示了如何使用CrewAI库快速构建和运行一个简单的协作团队，以完成特定任务。
- 首先定义了一个搜索引擎工具并设置了本地语言模型与嵌入器。
- 接着创建了两个代理角色——高级研究员和作家，并为他们分别分配了研究和技术写作的任务。这些任务包括识别领域内的最新趋势以及撰写相关文章。
- 最后，将这两个代理和任务组织成一个团队，并按照顺序执行流程以完成整个项目。

```mermaid
graph LR
    C[创建高级研究员代理]
    C --> D[分配研究任务给研究员]
    E[创建作家代理]
    E --> F[分配写作任务给作家]
    F --> G[组织团队并启动流程]
    D --> G
    G --> H{是否完成？}
    H -->|否| I[继续执行任务]
    H -->|是| J[输出结果]

```


首先，这里定义一个互联网查询工具，API需要自行获取

In [1]:
from api_key import SERPER_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key

from crewai_tools import SerperDevTool
search_tool = SerperDevTool(n_results=1)

为了让crewai在本地运行，通过以下代码设置llm及embedding

In [2]:
from crewai.llm import LLM
llm=LLM(
    model="ollama_chat/qwen2.5:latest", 
    base_url="http://localhost:11434")

embedder={
    "provider": "ollama",
    "config": {
        "model": 'quentinz/bge-large-zh-v1.5:latest',
        "base_url": 'http://localhost:11434'
    }
}

这里假设团队由2个角色组成，1个是相关领域的研究者、1个作家，通过研究者的研究成功，作家编写文章

In [3]:
# 设置litellm，不然日志过长
import litellm
litellm.set_verbose=False


from crewai import Agent
# 创建一个带有内存和冗长模式的高级研究员代理人
researcher = Agent(
    role='Senior Researcher',
    goal='在 {topic} 领域发现突破性技术',
    verbose=True,
    memory=False,
    backstory=(
        "出于好奇心驱使，您处于创新的最前沿，渴望探索和分享可能改变世界的知识。"
    ),
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# 创建一个作家代理人，具有自定义工具和委派能力
writer = Agent(
    role='Writer',
    goal='讲述关于 {topic} 的引人入胜的科技故事',
    verbose=True,
    memory=False,
    backstory=(
        "擅长简化复杂话题，您撰写引人入胜的叙述，吸引人并教育他人，在易于理解的方式中揭示新的发现。"
    ),
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

为团队的不同角色分配任务，指定执行任务可以使用的工具

In [4]:
from crewai import Task

# 研究任务
research_task = Task(
  description=(
    "识别 {topic} 领域的下一个大趋势。"
    "专注于识别利弊以及整体叙述。"
    "您的最终报告应清楚地阐明关键点、市场机会和潜在风险。"
  ),
  expected_output='一份关于最新人工智能趋势的全面报告，共 3 段。',
  tools=[search_tool],
  agent=researcher,
)

# 具有语言模型配置的写作任务
write_task = Task(
  description=(
    "撰写一篇关于 {topic} 的有见地的文章。"
    "关注最新趋势及其对行业的影响。"
    "这篇文章应易于理解、引人入胜且积极向上。"
  ),
  expected_output='一篇关于 {topic} 进展的 4 段文章，格式为 markdown。',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='关于"{topic}的进展".md'  # 输出定制的示例
)

通过角色（Agent）及任务（Task）定义一个团队，并为这个团队指定以下规则或配置：

- process：规定任务执行的顺序，可以假定为企业的标准SOP，比如申请购买东西，按照填写申请单->主管批准->采购购买->财务付款..等标准流程
- memory：为团队配备存储系统，可以记录每个成员的活动记录

In [5]:
from crewai import Crew, Process

# 以一些增强配置形成以技术为中心的团队
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential,  # 可选：顺序任务执行是默认设置
    memory=True,
    cache=False,
    max_rpm=100,
    share_crew=False,
    verbose=True,
    manager_llm=[llm],
    embedder=embedder
)

In [6]:
# 让团队开始按照既定流程工作
result = crew.kickoff(inputs={'topic': '医疗人工智能'})
print(result)

# Agent: Senior Researcher
## Task: 识别 医疗人工智能 领域的下一个大趋势。专注于识别利弊以及整体叙述。您的最终报告应清楚地阐明关键点、市场机会和潜在风险。


# Agent: Senior Researcher
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"recent breakthroughs in medical AI\"}"
## Tool Output: 
{'searchParameters': {'q': 'recent breakthroughs in medical AI', 'type': 'search', 'num': 1, 'engine': 'google'}, 'organic': [{'title': 'Top 6 AI Breakthroughs in Healthcare - Docwire News', 'link': 'https://docwirenews.com/post/the-top-6-ai-breakthroughs-in-healthcare', 'snippet': 'Top 6 AI Breakthroughs in Healthcare · Google Creates AI that Detects Lung Cancer Better than Doctors · Discovering Causes of Autism in Uncharted DNA · How do ...', 'position': 1, 'sitelinks': [{'title': 'Google Creates Ai That...', 'link': 'https://docwirenews.com/post/the-top-6-ai-breakthroughs-in-healthcare#:~:text=Google%20Creates%20AI%20that%20Detects%20Lung%20Cancer%20Better%20than%20Doctors'}, {'title': 'Researchers Use Machine...', 'link': 'https://

c:\Users\wushaogui\miniconda3\envs\crewai\lib\site-packages\pydantic\_internal\_generate_schema.py:297: PydanticDeprecatedSince20: `json_encoders` is deprecated. See https://docs.pydantic.dev/2.10/concepts/serialization/#custom-serializers for alternatives. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider L

以下是团队的最终工作成果，由作家输出，并且指定输出为`关于”医疗人工智能“的进展.md`

![alt text](doc/image.png)